## Role Prompting & Prompt Templates

*(Coding along with the [Anthropic's Prompt Engineering Interactive Tutorial](https://github.com/anthropics/courses/tree/master/prompt_engineering_interactive_tutorial/Anthropic%201P) which can be found on Anthropic's courses GitHub repo)*

### Basic Setup

In [4]:
# https://github.com/anthropics/courses/blob/master/prompt_engineering_interactive_tutorial/Anthropic%201P/01_Basic_Prompt_Structure.ipynb
from anthropic import Anthropic
import pandas as pd
import re

anthropic_api_key = pd.read_csv("~/tmp/anthropic/anthropic-key-1.txt", sep=" ", header=None)[0][0]
print("Don't be a fool and sent your api key to github")

# instantiating the client
client = Anthropic(api_key=anthropic_api_key)
MODEL_NAME="claude-3-5-sonnet-20241022"

Don't be a fool and sent your api key to github


In [5]:
# helper function that sends a prompt to Claude and returns Claude's generated response
def get_completion(prompt: str, system_prompt=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt}
        ]
    )
    
    return message.content[0].text

## Role Prompting

__Role Prompting:__

- To give Claude context, it is possible to prompt it to inhabit a specific role. 
- The more detail to the role context, the better.
- Priming Claude with a role can improve it's performance in a variety of fields. 
- Role prompting can also change the style, tone, and manner of Claude's response.
- Role prompting can happen either in the system prompt or as part of the User message turn.

### Examples

In [6]:
# (1) no role prompting
# Claude provides a straightforward and non-stylized answer
PROMPT = "In one sentence, what do you think about skateboarding?"
print(get_completion(PROMPT))

Skateboarding is an impressive blend of athleticism, creativity, and self-expression that has evolved from a counter-culture activity into a respected sport and art form.


In [8]:
# (2) same user question, now with role prompting
SYSTEM_PROMPT = "You are a cat."
PROMPT = "In one sentence, what do you think about skateboarding?"
print(get_completion(PROMPT, SYSTEM_PROMPT))

*Flicks tail dismissively* Those noisy rolling things look dangerous and completely inferior to gracefully leaping from high places, which is what any sensible creature would do.


#### Role prompting can make Claude better at performing math or logic tasks

In [11]:
# in the example below, there is a definitive correct answer, which is yes
# however, Claude gets it wrong and thinks it lacks information, which it doesn't
PROMPT = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"
print(get_completion(PROMPT)) # Claude is actually getting it right wich differs from the tutorial result

Let me solve this step by step:

1) We know:
   * Jack is looking at Anne
   * Anne is looking at George
   * Jack is married
   * George is unmarried
   * Anne's marital status is unknown

2) To answer if a married person is looking at an unmarried person, we need to consider two cases:

   Case 1: If Anne is married
   * Jack (married) is looking at Anne (married)
   * Anne (married) is looking at George (unmarried)
   * In this case, YES - a married person (Anne) is looking at an unmarried person (George)

   Case 2: If Anne is unmarried
   * Jack (married) is looking at Anne (unmarried)
   * Anne (unmarried) is looking at George (unmarried)
   * In this case, YES - a married person (Jack) is looking at an unmarried person (Anne)

3) In both cases, regardless of Anne's marital status, a married person is looking at an unmarried person.

Therefore, the answer is YES, a married person is definitely looking at an unmarried person.


In [10]:
# with role assignment, Claude gets it right
SYSTEM_PROMPT = "You are a logic bot designed to answer complex logic problems."
PROMPT = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"
print(get_completion(PROMPT, SYSTEM_PROMPT))

Let me solve this step by step.

1) We know:
   * Jack is looking at Anne
   * Anne is looking at George
   * Jack is married
   * George is unmarried
   * Anne's marital status is unknown

2) To answer if a married person is looking at an unmarried person, we need to consider both:
   * Jack looking at Anne
   * Anne looking at George

3) Let's analyze each case:
   * Jack (married) is looking at Anne (unknown)
   * Anne (unknown) is looking at George (unmarried)

4) For Jack looking at Anne:
   * If Anne is married → married person looking at married person
   * If Anne is unmarried → married person looking at unmarried person

5) For Anne looking at George:
   * If Anne is married → married person looking at unmarried person
   * If Anne is unmarried → unmarried person looking at unmarried person

6) Conclusion:
   * If Anne is married → Yes (Anne looking at George)
   * If Anne is unmarried → Yes (Jack looking at Anne)

Therefore, regardless of Anne's marital status, the answer is 

### Exercise: Math Correction

In [12]:
# System prompt - if you don't want to use a system prompt, you can leave this variable set to an empty string
# the tutorial says: below, Claude incorrectly assesses the math problem as correctly solved
# this is wrong, Claude gets it right without system prompt
SYSTEM_PROMPT = ""

# Prompt
PROMPT = """Is this equation solved correctly below?

2x - 3 = 9
2x = 6
x = 3"""

# Get Claude's response
response = get_completion(PROMPT, SYSTEM_PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    if "incorrect" in text or "not correct" in text.lower():
        return True
    else:
        return False

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

No, this equation is not solved correctly. Let me show you the correct solution:

2x - 3 = 9

First, add 3 to both sides to isolate the term with x:
2x - 3 + 3 = 9 + 3
2x = 12

Then, divide both sides by 2:
2x ÷ 2 = 12 ÷ 2
x = 6

The error in the original solution was going from 2x - 3 = 9 to 2x = 6. This step skipped the proper addition of 3 to both sides and incorrectly changed 9 to 6.

You can verify the answer by plugging x = 6 back into the original equation:
2(6) - 3 = 9
12 - 3 = 9
9 = 9 ✓

--------------------------- GRADING ---------------------------
This exercise has been correctly solved: True


## Prompt Templates: Separating Data and Instructions

__Writing a substitutable prompt template:__

- We want a prompt templates that can be modified later with additional input data before submitting to Claude.
- This is useful if we want Claude to do the same thing repeatedly but with different data each time: __prompt templates simplify repetitive tasks.__ 
- We can achieve this by separating the fixed skeleton of the prompt from variable user input.
- We do this substitution using variables and f-strings, but you can also do it with the format() method.
- It is very important to make sure Claude knows where variables start and end: __wrap the input in XML tags__.

### Example: Asking Claude to act as an animal noise generator

The full prompt submitted to Claude is a `PROMPT_TEMPLATE` substituted with the input that replaces the `ANIMAL` placeholder via an f-string when we print out the full prompt.

In [13]:
# Variable content
ANIMAL = "Cow"

# Prompt template with a placeholder for the variable content
PROMPT = f"I will tell you the name of an animal. Please respond with the noise that animal makes. {ANIMAL}"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

--------------------------- Full prompt with variable substutions ---------------------------
I will tell you the name of an animal. Please respond with the noise that animal makes. Cow

------------------------------------- Claude's response -------------------------------------
Moo!


Example for surrounding the user input sentences in XML tags. This shows Claude where the input data begins and ends despite the misleading hyphen before `Each is about an animal, like rabbits`.

In [18]:
# Variable content
SENTENCES = """- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs"""

# Prompt template with a placeholder for the variable content
# setting the placeholder in XML tags
# otherwise Claude gets confused by the hyphen before "Each" and takes it as part of the list
PROMPT = f"""Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
<sentences>{SENTENCES}</sentences>"""

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))
     

--------------------------- Full prompt with variable substutions ---------------------------
Below is a list of sentences. Tell me the second item on the list.

- Each is about an animal, like rabbits.
<sentences>- I like how cows sound
- This sentence is about spiders
- This sentence may appear to be about dogs but it's actually about pigs</sentences>

------------------------------------- Claude's response -------------------------------------
The second sentence in the list is: "This sentence is about spiders"


### Exercise: Dog Question with Typos

In [22]:
# the messy and mistake-ridden writing is intentional here
# so you can see how Claude reacts to such mistakes

# Variable content
QUESTION = "ar cn brown?"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hia its me i have a q about dogs jkaerjv <question>{QUESTION}</question> jklmvca tx it help me muhch much atx fst fst answer short short tx"

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("brown", text.lower()))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

--------------------------- Full prompt with variable substutions ---------------------------
Hia its me i have a q about dogs jkaerjv <question>ar cn brown?</question> jklmvca tx it help me muhch much atx fst fst answer short short tx

------------------------------------- Claude's response -------------------------------------
Yes, dogs can be brown. Brown is a very common color for dogs.

------------------------------------------ GRADING ------------------------------------------
This exercise has been correctly solved: True
